In [7]:
import torch

In [15]:
inputs = torch.nn.Embedding( 4, 8 )

In [16]:
inputs = inputs.weight
inputs 

Parameter containing:
tensor([[-0.0510,  0.5949, -0.2721, -0.4530,  0.9421, -1.3259,  1.4078,  0.5397],
        [ 0.2901, -0.6298,  0.2870, -0.3161, -0.1634,  1.7088, -0.0341,  1.2769],
        [ 1.9066, -1.6460, -0.0770,  1.4417, -0.2897,  1.5766,  0.2341,  0.0040],
        [-0.9596,  1.1139,  0.1864,  0.1311,  0.0153,  1.6172, -0.3318, -0.8524]],
       requires_grad=True)

In [17]:
inputs = inputs.data
inputs

tensor([[-0.0510,  0.5949, -0.2721, -0.4530,  0.9421, -1.3259,  1.4078,  0.5397],
        [ 0.2901, -0.6298,  0.2870, -0.3161, -0.1634,  1.7088, -0.0341,  1.2769],
        [ 1.9066, -1.6460, -0.0770,  1.4417, -0.2897,  1.5766,  0.2341,  0.0040],
        [-0.9596,  1.1139,  0.1864,  0.1311,  0.0153,  1.6172, -0.3318, -0.8524]])

In [41]:
# Set dimensions
d_in = 8
d_out = 6
# create weight matrices
W_q = torch.nn.Parameter( torch.randn( (d_in, d_out), requires_grad=False ) )
W_k = torch.nn.Parameter( torch.randn( (d_in, d_out), requires_grad=False ) )
W_v = torch.nn.Parameter( torch.randn( (d_in, d_out), requires_grad=False ) )

In [42]:
# Choose an input vector and transform it into our queryvector using W_q
query = inputs[2] @ W_q
query

tensor([ 1.8784,  1.7931, -3.5117,  0.7535, -2.6289,  6.0498],
       grad_fn=<SqueezeBackward4>)

In [43]:
# calculate attention scores using the keys generated by W_k
keys = inputs @ W_k
values = inputs @ W_v
print("Keys:" , keys)
print("Values:" , values)

Keys: tensor([[-1.2708, -1.5589,  1.9769,  3.7683,  0.1168,  0.1325],
        [-2.4639, -0.9638,  0.3695, -0.6698,  0.1940,  3.2563],
        [-3.5440, -0.9084, -3.1347, -4.7587, -6.9262,  3.0106],
        [ 4.6047,  2.1761,  2.0887, -1.0419,  1.4236, -0.5873]],
       grad_fn=<MmBackward0>)
Values: tensor([[ 4.6588, -0.6482,  1.8365,  2.1508,  4.9165,  1.4143],
        [-5.2708,  1.4463,  2.6178, -1.5499, -3.8946, -1.7138],
        [-5.4761, -1.8431, -2.8577, -1.7862, -5.3302, -4.7337],
        [-2.4833,  0.1127, -0.2006, -1.4244, -1.7792,  1.0734]],
       grad_fn=<MmBackward0>)


In [44]:
attention_scores = query @ keys.T
attention_scores

tensor([-8.7906, 11.0315, 35.5578, -2.8643], grad_fn=<SqueezeBackward4>)

In [45]:
attention_weights = torch.softmax( attention_scores, dim=-1 )
attention_weights

tensor([5.4921e-20, 2.2302e-11, 1.0000e+00, 2.0582e-17],
       grad_fn=<SoftmaxBackward0>)

In [46]:
attention_weights.sum()

tensor(1., grad_fn=<SumBackward0>)

In [47]:
context_vector = attention_weights @ values
context_vector

tensor([-5.4761, -1.8431, -2.8577, -1.7862, -5.3302, -4.7337],
       grad_fn=<SqueezeBackward4>)